# Imports

In [1]:
%load_ext autoreload

In [2]:
# Load the Pandas libraries with alias 'pd'
import datetime

import numpy as np
import pandas as pd
from bokeh.io import output_notebook
from bokeh.layouts import grid
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource, LinearAxis, Range1d
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10_6, Category20_20
from bokeh.plotting import figure, show

from covid19.covid import (
    get_all_states_data,
    get_date_date_cases_greater_than,
    get_dia_de_contaminacao_array,
    get_state_data,
)
from covid19.scrap import get_covid_data, read_datafile_from_disc

output_notebook(hide_banner=True)

# Lendo os dados

In [3]:
# data = pd.read_excel("DT_PAINEL_COVIDBR_20200510.xlsx")
data = read_datafile_from_disc("DT1_PAINEL_COVIDBR_20200512.xlsx")

# Entendendo os dados e separando dados do Brasil todo e de cada estado

As primeiras linhas possuem dados para o Brasil todo e a região é "Brasil".

In [4]:
print("Shape do dataset todo", data.shape)
data.head()

Shape do dataset todo (69650, 14)


,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,0,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,0,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-03-01,10,210147125.0,2,0,NaN,NaN


Vamos adicionar uma coluna contendo o dia da semana. Como a coluna "data" possui objetos do tipo `datetime.date` basta usarmos a função `apply` de um dataframe.

In [5]:
weekDays = ("Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado",
            "Domingo")
data["diaDaSemana"] = data.apply(lambda x: weekDays[x.data.weekday()], axis=1)

Vamos separar o dataset em dois: um para o Brasil todo e outro para cada cidade.

In [6]:
data_brasil = data.query("regiao=='Brasil'").copy()

print("Shape do dataset do Brasil", data_brasil.shape)
# data_cidades = data.query("regiao!='Brasil'").copy()

Shape do dataset do Brasil (77, 15)


In [7]:
data_estados = get_all_states_data(data)
print("Shape do dataset dos estados", data_estados.shape)
print(data_estados.shape[0] / 27)

Shape do dataset dos estados (1626, 15)
60.22222222222222


# Primeiros casos

In [8]:
# data_brasil = data.groupby("data").sum()

print(f"Data do primeiro caso: {get_date_date_cases_greater_than(data, 0)}")
print(
    f"Data com mais de 10 casos: {get_date_date_cases_greater_than(data, 10)}")
print(
    f"Data com mais de 100 casos: {get_date_date_cases_greater_than(data, 100)}"
)
print(
    f"Data com mais de 1000 casos: {get_date_date_cases_greater_than(data, 1000)}"
)

Data do primeiro caso: 2020-02-26
Data com mais de 10 casos: 2020-03-06
Data com mais de 100 casos: 2020-03-14
Data com mais de 1000 casos: 2020-03-21


In [9]:
get_state_data(data, "CE").head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana
12679,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-03-17,12,9132078.0,5,0,NaN,NaN,Terça
12680,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-03-18,12,9132078.0,9,0,NaN,NaN,Quarta
12681,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-03-19,12,9132078.0,20,0,NaN,NaN,Quinta
12682,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-03-20,12,9132078.0,55,0,NaN,NaN,Sexta
12683,Nordeste,CE,NaN,23,NaN,NaN,NaN,2020-03-21,12,9132078.0,68,0,NaN,NaN,Sábado


# Vamos adicionar as colunas "casosNovos", "obitosNovos" e "diaDeContaminacao"

Primeiro adicionamos no dataframe do Brasil todo.

In [10]:
# Calcula casos novos a partir dos casos acumulados. Note que o primeiro valor na coluna "casosNovos" corresponde ao total de casos acumulados na primeira entrada
data_brasil["casosNovos"] = np.diff(data_brasil.casosAcumulado, prepend=0)
data_brasil["obitosNovos"] = np.diff(data_brasil.obitosAcumulado, prepend=0)

data_brasil["diasDeContaminacao"] = get_dia_de_contaminacao_array(data_brasil)

print(np.all(np.cumsum(data_brasil.casosNovos) == data_brasil.casosAcumulado))

True


In [11]:
# sigla = "CE"
# # print(data_estados.loc[data_estados.estado == sigla, "diasDeContaminacao"].shape)

# ce = data_estados[data_estados.estado == sigla]
# primeiro_dia_de_contaminacao = get_date_date_cases_greater_than(ce)
# bool_array = ce.data >= primeiro_dia_de_contaminacao

# num_dates = ce.shape[0]
# num_contamination_dates = sum(bool_array)
# num_contamination_dates

# np.concatenate([np.zeros(num_dates - num_contamination_dates, dtype=int), np.arange(1, num_contamination_dates+1, dtype=int)]).shape
# # get_dia_de_contaminacao_array(data_estados[data_estados.estado == sigla])

In [12]:
estados = data_estados.estado.unique()

# Calcula novos casos a partir dos casos acumulados. Note que o primeiro valor na coluna "casosNovos" corresponde ao total de casos acumulados na primeira entrada
data_estados["casosNovos"] = np.zeros(data_estados.shape[0], dtype=int)
data_estados["diasDeContaminacao"] = np.zeros(data_estados.shape[0], dtype=int)
for sigla in estados:
    data_esse_estado = data_estados[data_estados.estado == sigla]
    data_estados.loc[data_estados.estado == sigla,
                     "casosNovos"] = np.diff(data_esse_estado.casosAcumulado,
                                             prepend=0)
    data_estados.loc[data_estados.estado == sigla,
                     "obitosNovos"] = np.diff(data_esse_estado.obitosAcumulado,
                                              prepend=0)
    data_estados.loc[data_estados.estado == sigla,
                     "diasDeContaminacao"] = get_dia_de_contaminacao_array(
                         data_estados[data_estados.estado == sigla])

data_estados

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,obitosAcumulado,Recuperadosnovos,emAcompanhamentoNovos,diaDaSemana,casosNovos,diasDeContaminacao,obitosNovos
887,Centro-Oeste,MS,NaN,50,NaN,NaN,NaN,2020-03-16,12,2778986.0,2,0,NaN,NaN,Segunda,2,1,0.0
888,Centro-Oeste,MS,NaN,50,NaN,NaN,NaN,2020-03-17,12,2778986.0,4,0,NaN,NaN,Terça,2,2,0.0
889,Centro-Oeste,MS,NaN,50,NaN,NaN,NaN,2020-03-18,12,2778986.0,7,0,NaN,NaN,Quarta,3,3,0.0
890,Centro-Oeste,MS,NaN,50,NaN,NaN,NaN,2020-03-19,12,2778986.0,7,0,NaN,NaN,Quinta,0,4,0.0
891,Centro-Oeste,MS,NaN,50,NaN,NaN,NaN,2020-03-20,12,2778986.0,9,0,NaN,NaN,Sexta,2,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69645,Sul,RS,NaN,43,NaN,NaN,NaN,2020-05-08,19,11377239.0,2447,91,NaN,NaN,Sexta,318,60,1.0
69646,Sul,RS,NaN,43,NaN,NaN,NaN,2020-05-09,19,11377239.0,2493,95,NaN,NaN,Sábado,46,61,4.0
69647,Sul,RS,NaN,43,NaN,NaN,NaN,2020-05-10,20,11377239.0,2542,97,NaN,NaN,Domingo,49,62,2.0
69648,Sul,RS,NaN,43,NaN,NaN,NaN,2020-05-11,20,11377239.0,2576,105,NaN,NaN,Segunda,34,63,8.0


# Plot dados para o Brasil

In [13]:
# variable_to_plot_vec = np.array(['casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos'])
variable_to_plot_vec = np.array(['casosAcumulado', 'obitosAcumulado'])

# We only want values from this date in the plot
date_threshold = get_date_date_cases_greater_than(data, 100)

In [14]:
source_brasil = ColumnDataSource(data_brasil)
fig = figure(title="Covid-19 no Brasil",
             x_axis_type='datetime',
             width=1200,
             height=400,
             x_axis_label="Data",
             y_axis_label="Casos Acumulados")
fig.line("data",
         "casosAcumulado",
         source=source_brasil,
         color=Category10_6[0],
         legend_label="Acumulados")
fig.circle("data",
           "casosAcumulado",
           source=source_brasil,
           color=Category10_6[0],
           legend_label="Acumulados")
fig.line("data",
         "obitosAcumulado",
         source=source_brasil,
         color=Category10_6[1],
         legend_label="Óbitos")
fig.circle("data",
           "obitosAcumulado",
           source=source_brasil,
           color=Category10_6[1],
           legend_label="Óbitos")
fig.legend.location = "top_left"
fig.legend.click_policy = 'hide'
fig.add_tools(
    HoverTool(
        tooltips=[("Casos Acumulados",
                   "@casosAcumulado"), ("Óbitos Acumulados", "@obitosAcumulado"
                                       ), ("Semana Epidemiológica",
                                           "@semanaEpi")]))

show(fig)

## Plot de vários estados por data

In [15]:
# state_abrv_vec = np.array(['CE', 'DF', 'PE', 'RJ', 'SP', 'BA', 'AM', 'MS', 'RN'])
state_abrv_vec = np.array(['CE', 'DF', 'RJ', 'SP', 'AM'])


def get_plot(column_name, y_axis_label="Label", width=1200, height=400):

    fig = figure(title="Covid-19 nos estados",
                 x_axis_type='datetime',
                 width=width,
                 height=height,
                 x_axis_label="Data",
                 y_axis_label=y_axis_label)
    for i, sigla in enumerate(state_abrv_vec):
        state_data = get_state_data(data_estados, sigla)
        source = ColumnDataSource(state_data)
        fig.line("data",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category10_6[i])
        fig.circle("data",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category10_6[i])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[("Estado", "@estado"), ("Dia da Semana", "@diaDaSemana"),
                      ("Casos Acumulados",
                       "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
width = 650
height = 300
plot_casosAcumulados = get_plot("casosAcumulado",
                                y_axis_label="Casos Acumulados",
                                width=width,
                                height=height)
plot_casosNovos = get_plot("casosNovos",
                           y_axis_label="Casos Novos",
                           width=width,
                           height=height)
plot_obitosAcumulados = get_plot("obitosAcumulado",
                                 y_axis_label="Óbitos Acumulados",
                                 width=width,
                                 height=height)
plot_obitosNovos = get_plot("obitosNovos",
                            y_axis_label="Óbitos Novos",
                            width=width,
                            height=height)

show(
    grid([[plot_casosAcumulados, plot_casosNovos],
          [plot_obitosAcumulados, plot_obitosNovos]]))

## Plot de vários estados por dia de contaminação


In [16]:
# Estados com crescimento mais rápido por dia de contaminação
estados_crescimento_rapido = ["CE", "SP", "AM", "PE", "RJ"]


def get_plot_por_dia_de_contaminacao(column_name,
                                     y_axis_label="Label",
                                     width=1200,
                                     height=400):
    fig = figure(
        title=
        "Covid-19 nos 5 estados com crescimento mais rápido por dia de contaminação",
        width=width,
        height=height,
        x_axis_label="Dia de Contaminação",
        y_axis_label=y_axis_label)

    for i, sigla in enumerate(estados_crescimento_rapido):
        state_data = get_state_data(data_estados, sigla)
        source = ColumnDataSource(state_data)
        fig.line("diasDeContaminacao",
                 column_name,
                 source=source,
                 legend_label=sigla,
                 color=Category20_20[i % 20])
        fig.circle("diasDeContaminacao",
                   column_name,
                   source=source,
                   legend_label=sigla,
                   color=Category20_20[i % 20])

    fig.legend.location = "top_left"
    #     fig.legend.title = "Estado"
    fig.legend.click_policy = 'hide'
    fig.add_tools(
        HoverTool(
            tooltips=[(
                "Estado",
                "@estado"), ("Dias de contaminação", "@diasDeContaminacao"),
                      ("Dia da Semana",
                       "@diaDaSemana"), (
                           "Casos Acumulados",
                           "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


plot_casosAcumulados_por_dia_de_contaminacao = get_plot_por_dia_de_contaminacao(
    "casosAcumulado", y_axis_label="Casos Acumulados", width=1200, height=500)
# plot_casosNovos_por_dia_de_contaminacao = get_plot_por_dia_de_contaminacao("casosNovos", y_axis_label="Casos Novos", width=width, height=height)
show(plot_casosAcumulados_por_dia_de_contaminacao)

<span style="color:red; font-weight: 700; font-size:20px;">A parte preocupande é que o Ceará tem um rítimo de crescimento tão rápido ou levemente maior que São Paulo. <br>
    A diferença é que o primeiro caso no Ceará foi 20 dias depois do primeiro caso em São Paulo.</span>

# Verificando se estamos saindo da crise

Um bom gráfico para saber se estamos saindo da crise é o de casos novos por casos existentes (ambos em escala log). Enquanto estivermos na epidemia o gráfico aproxima-se de uma reta. Quando começarmos a sair da epidemia o gráfico cai.

In [17]:
def get_plot_casos_novos_por_casos_acumulados(df, width=1200, height=400):
    fig = figure(title="Casos novos por casos acumulados",
                 width=width,
                 height=height,
                 x_axis_type="log",
                 y_axis_type="log",
                 x_axis_label="Casos Acumulados (log)",
                 y_axis_label="Casos Novos (log)")
    fig.line('casosAcumulado', "casosNovos", source=df)
    fig.circle('casosAcumulado', "casosNovos", source=df)
    fig.add_tools(
        HoverTool(
            tooltips=[("Dia da Semana",
                       "@diaDaSemana"), (
                           "Casos Acumulados",
                           "@casosAcumulado"), ("Casos Novos", "@casosNovos"),
                      ("Óbitos Acumulados",
                       "@obitosAcumulado"), ("Óbitos Novos", "@obitosNovos")]))
    return fig


plot_casos_novos_por_acumulados = get_plot_casos_novos_por_casos_acumulados(
    data_brasil)
show(plot_casos_novos_por_acumulados)